In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sys
from scipy.special import erfcinv as erfcinv
import tqdm as tqdm
import time
from models import gauss_func,gp_forward_model
import matplotlib.pyplot as plt
from plotting import mole_fraction_plot
from read_settings import read_settings

In [ ]:
grid,source,atm,obs = read_settings('../site_settings.yml')

## Compute GP Solution on 3-D Grid

In [ ]:
mass_density_enh = gp_forward_model(source=source,atm=atm,grid=grid)
mole_fraction_enh = mass_density_enh * atm['scaling']
mole_fraction = atm['background'] + mole_fraction_enh
is_mf_error = obs['is_noise']/100.*np.random.randn(*mole_fraction_enh.shape)*atm['background']
noisy_mole_fraction = mole_fraction+is_mf_error

x = grid['advect_axis']
y = grid['crosswind_axis']
z = grid['vertical_axis']

## Surface Mole Fraction Plots

In [ ]:
lev=0
mole_fraction_plot(mole_fraction=mole_fraction[lev],plot_lims=[1.8,3.5],x=x,y=y,x_slices=[1000,2000,3000,4000],title=f'CH$_4$ (ppm) at {z[lev]}m above surface')
mole_fraction_plot(mole_fraction=is_mf_error[lev],plot_lims=[-4e-3,4e-3],x=x,y=y,x_slices=[1000,2000,3000,4000],title=f'CH$_4$ Noise (ppm) at {z[lev]}m above surface',cmap=plt.cm.RdBu_r)
mole_fraction_plot(mole_fraction=noisy_mole_fraction[lev],plot_lims=[1.8,3.5],x=x,y=y,x_slices=[1000,2000,3000,4000],title=f'Noisy CH$_4$ (ppm) at {z[lev]}m above surface')

## Vertical Profile of Mole Fraction Plots

In [ ]:
y_slice = 0
slice_ind = np.argwhere(np.abs(y-y_slice)<1e-3)[0][0]
mole_fraction_plot(mole_fraction=mole_fraction[:,slice_ind,:],plot_lims=[1.8,5],x=x,y=z,x_slices=[500,1000,2000,4000],ylabel=f'Vertical Direction (km)',title=f'Vertical CH$_4$ (ppm) along y={y_slice}')


## Compute Column Average Mole Fraction 

In [ ]:
col_mass_density_enh = mass_density_enh.sum(0)*grid['dp']
column_avg_mole_fraction = mole_fraction.sum(0)*np.abs(grid['dp'])/(grid['p'][0]-grid['p'][-1]) + atm['background']*np.abs(grid['dp'])/grid['p'][-1]
col_mf_error = obs['col_noise']/100*np.random.randn(*column_avg_mole_fraction.shape)*atm['background']
noisy_column_avg_mole_fraction = column_avg_mole_fraction + col_mf_error

## Column Avg Mole Fraction Plots

In [ ]:
mole_fraction_plot(mole_fraction=column_avg_mole_fraction[:],plot_lims=[1.8,2.3],x=x,y=y,x_slices=[1000,2000,3000,4000],title=f'XCH$_4$ (ppm)')
mole_fraction_plot(mole_fraction=col_mf_error[:],plot_lims=[-0.3,0.3],x=x,y=y,x_slices=[1000,2000,3000,4000],title=f'XCH$_4$ Noise (ppm)',cmap=plt.cm.RdBu_r)
mole_fraction_plot(mole_fraction=noisy_column_avg_mole_fraction[:],plot_lims=[1.6,2.3],x=x,y=y,x_slices=[1000,2000,3000,4000],title=f'Noisy XCH$_4$ (ppm)')